## Problématique : l'utilisateur va t-il résilier dans les 30 jours ?

Objectif : Construire un modèle qui se base sur les 48 premières heures d'utilisation de l'application et qui prédit si un utilisateur (qui paie durant les 48 premières heures) va résilier ou non


### Import des librairies

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

### Visualisation des données

In [3]:
df = pd.read_parquet('app_ios_data_20220901_20230113.parquet', engine='pyarrow')
df.head()
print("Taille du dataframe : ", df.shape)

Taille du dataframe :  (17487321, 47)


### Données des 48 premières heures

In [4]:
df48 = df[df["created_at"] < df["installed_at"] + pd.DateOffset(days=2)]
### On sélectionne seulement les données d'une durée de 48 heures après l'installation de l'application
df48.head()

,amplitude_id,client_event_time,client_upload_time,country,device_family,device_model,event_id,created_at,event_type,language,...,push_notification_allowed,premium,no_of_custom_routines_saved,daily_reminder,days_completed_count,last_stretch_time,active_streak_count,longest_streak_count,apple_health,att
0,532864260650,2022-12-24 06:07:19.881,2022-12-24 06:07:19.906,United Kingdom,Apple iPhone,iPhone XR,1,2022-12-24 06:07:19.881,session_start,English,...,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,nan,NaN
1,532864288194,2022-12-24 06:07:23.963,2022-12-24 06:07:23.986,Australia,Apple iPhone,iPhone 8,1,2022-12-24 06:07:23.963,session_start,English,...,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,nan,NaN
2,532864260650,2022-12-24 06:07:24.875,2022-12-24 06:07:49.965,United Kingdom,Apple iPhone,iPhone XR,2,2022-12-24 06:07:24.875,onboarding_start,English,...,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,nan,NaN
3,532864260650,2022-12-24 06:07:25.966,2022-12-24 06:07:49.965,United Kingdom,Apple iPhone,iPhone XR,4,2022-12-24 06:07:25.966,onboarding_page_view,English,...,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,nan,NaN
4,532864260650,2022-12-24 06:07:26.192,2022-12-24 06:07:49.965,United Kingdom,Apple iPhone,iPhone XR,6,2022-12-24 06:07:26.192,first_app_open,English,...,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,nan,NaN


### Suppression des non payeurs dans les 48 premières heures

In [5]:
df48["paying"] = df["paying"].replace(["nan", "true"], [0,1]) ### normalisation de la colonne paying
paying48 = df48[["amplitude_id", "paying"]].groupby("amplitude_id").max()
### on obtient un dataframe qui, pour chaque utilisateur, nous indique s'il a payé durant les 48 premières heures
payers_id = list(paying48[paying48["paying"] == 1].index) 
### on récupère l'id des utilisateurs qui ont payé *payeurs*
payers = df48[df48["amplitude_id"].isin(payers_id)] 
### on considère que les données de ces payeurs

/var/folders/dd/0hw4ljv901vct22jhgypdgd00000gn/T/ipykernel_41846/3870606840.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df48["paying"] = df["paying"].replace(["nan", "true"], [0,1]) ### normalisation de la colonne paying


### Suppression des payeurs qui résilient dans les 48 premières heures
Il se peut que des utilisateurs qui paient dans les 48 premières heures résilient tout de suite après. On va donc les enlever des données car ce type d'utilisateurs ne nous intéresse pas.

In [6]:
not_cancelers_id = list(payers[payers["event_type"] != "rc_cancellation_event"]["amplitude_id"].unique()) 
### on récupère les ids de ceux qui ne résilient pas dans les 48 premières heures
payers = payers[payers["amplitude_id"].isin(not_cancelers_id)] 
### on considère que les données de ces payeurs pour l'entrainement du modèle

In [7]:
payers.head()

,amplitude_id,client_event_time,client_upload_time,country,device_family,device_model,event_id,created_at,event_type,language,...,push_notification_allowed,premium,no_of_custom_routines_saved,daily_reminder,days_completed_count,last_stretch_time,active_streak_count,longest_streak_count,apple_health,att
25,532651279569,2022-12-24 06:14:11.943,2022-12-24 06:14:26.265,Germany,Apple iPhone,iPhone 11 Pro Max,215,2022-12-24 06:14:11.943,page_view,English,...,true,true,0.0,true,NaN,NaN,NaN,NaN,nan,NaN
26,532651279569,2022-12-24 06:14:14.594,2022-12-24 06:14:26.265,Germany,Apple iPhone,iPhone 11 Pro Max,217,2022-12-24 06:14:14.594,page_view,English,...,true,true,0.0,true,NaN,NaN,NaN,NaN,nan,NaN
30,532866418590,2022-12-24 06:14:27.465,2022-12-24 06:14:57.532,United Arab Emirates,Apple iPhone,iPhone 11 Pro Max,24,2022-12-24 06:14:27.465,onboarding_tap_continue,English,...,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,nan,NaN
31,532866418590,2022-12-24 06:14:28.312,2022-12-24 06:14:57.532,United Arab Emirates,Apple iPhone,iPhone 11 Pro Max,26,2022-12-24 06:14:28.312,onboarding_page_view,English,...,nan,nan,NaN,nan,NaN,NaN,NaN,NaN,nan,NaN
32,532866418590,2022-12-24 06:14:46.128,2022-12-24 06:14:57.532,United Arab Emirates,Apple iPhone,iPhone 11 Pro Max,28,2022-12-24 06:14:46.128,ac_daily_reminder_activate,English,...,nan,nan,NaN,true,NaN,NaN,NaN,NaN,nan,NaN


In [8]:
payers["amplitude_id"].to_csv("payers48_id.csv", index = False)

### Récupération des données de payeurs des 30 jours après les 48 premières heures
On va maintenant regarder si ces utilisateurs résilient ou non grâce à l'événement **rc_cancellation_event**.

In [9]:
payers_id = list(payers["amplitude_id"].unique()) 
### récupération des ids des payeurs dans les 48 premières heures
df_payers = df[df["amplitude_id"].isin(payers_id)] 
### données totales des payeurs sur toute la période de leur utilisation de l'application

In [10]:
df_after48 = df_payers[(df_payers["client_event_time"] >= df_payers["installed_at"] + pd.DateOffset(days=2)) & 
                (df_payers["client_event_time"] <= df_payers["installed_at"] + pd.DateOffset(days=32))]
### on se restreint aux données sur une période de 30 jours après les 48 premières heures
df_after48.head()

,amplitude_id,client_event_time,client_upload_time,country,device_family,device_model,event_id,created_at,event_type,language,...,push_notification_allowed,premium,no_of_custom_routines_saved,daily_reminder,days_completed_count,last_stretch_time,active_streak_count,longest_streak_count,apple_health,att
494,525500759222,2022-12-22 04:24:31.068,2022-12-24 06:13:05.779,Canada,Apple iPhone,"iPhone14,7",577,2022-12-22 04:24:31.068,session_end,English,...,true,true,0.0,true,7.0,1.671683e+09,3.0,3.0,nan,NaN
495,525500759222,2022-12-24 06:13:05.725,2022-12-24 06:13:05.779,Canada,Apple iPhone,"iPhone14,7",578,2022-12-24 06:13:05.725,session_start,English,...,true,true,0.0,true,7.0,1.671683e+09,3.0,3.0,nan,NaN
496,525500759222,2022-12-24 06:13:05.725,2022-12-24 06:13:05.779,Canada,Apple iPhone,"iPhone14,7",579,2022-12-24 06:13:05.725,app_open,English,...,true,true,0.0,true,7.0,1.671683e+09,3.0,3.0,nan,NaN
507,525500759222,2022-12-24 06:13:08.308,2022-12-24 06:13:35.790,Canada,Apple iPhone,"iPhone14,7",580,2022-12-24 06:13:08.308,page_view,English,...,true,true,0.0,true,7.0,1.671683e+09,3.0,3.0,nan,NaN
508,525500759222,2022-12-24 06:13:11.201,2022-12-24 06:13:35.790,Canada,Apple iPhone,"iPhone14,7",581,2022-12-24 06:13:11.201,ac_content_click,English,...,true,true,0.0,true,7.0,1.671683e+09,3.0,3.0,nan,NaN


### Création de la variable cible
Une fois les données des payeurs sur une période de 30 jours après les 48 heures, on regarde si les utilisateurs ont déclenché l'événement **rc_cancellation_event**.

In [11]:
df_after48["is_rc_cancellation_event"] = (df_after48["event_type"] == "rc_cancellation_event")
df_after48["is_rc_cancellation_event"] = df_after48["is_rc_cancellation_event"].replace([False, True], [0, 1])
### on créé une nouvelle colonne binaire qui indique si l'événement est rc_cancellation_event
cancel = df_after48[["amplitude_id", "is_rc_cancellation_event"]].groupby("amplitude_id").max()
### on obtient un dataframe qui, pour chaque utilisateur, indique s'il a résilié ou non dans les 30 jours post 48h

/var/folders/dd/0hw4ljv901vct22jhgypdgd00000gn/T/ipykernel_41846/4035124632.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_after48["is_rc_cancellation_event"] = (df_after48["event_type"] == "rc_cancellation_event")
/var/folders/dd/0hw4ljv901vct22jhgypdgd00000gn/T/ipykernel_41846/4035124632.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_after48["is_rc_cancellation_event"] = df_after48["is_rc_cancellation_event"].replace([False, True], [0, 1])


In [12]:
cancel.head()

,is_rc_cancellation_event
amplitude_id,
454953384006,0
454959727343,0
454961916385,0
454961955839,1
454962783634,0


In [13]:
cancel.value_counts()

is_rc_cancellation_event
0                           9943
1                           2748
dtype: int64

In [14]:
cancel.reset_index().to_csv("cancels.csv", index = False)

### A vous de jouer !
Vous pouvez utiliser les données de **payers** pour les pré processer et les agréger afin de prédire si ce payeur va résilier ou non.

In [61]:
payers.head()

,amplitude_id,client_event_time,client_upload_time,country,device_family,device_model,event_id,created_at,event_type,language,...,push_notification_allowed,premium,no_of_custom_routines_saved,daily_reminder,days_completed_count,last_stretch_time,active_streak_count,longest_streak_count,apple_health,att
538,532579182624,2022-12-24 06:24:29.950,2022-12-24 06:24:56.095,United States,Apple iPhone,"iPhone14,2",93,2022-12-24 06:24:29.950,ac_content_click,English,...,true,true,NaN,true,1.0,1.671821e+09,1.0,1.0,nan,NaN
539,532579182624,2022-12-24 06:24:30.383,2022-12-24 06:24:56.095,United States,Apple iPhone,"iPhone14,2",95,2022-12-24 06:24:30.383,page_view,English,...,true,true,NaN,true,1.0,1.671821e+09,1.0,1.0,nan,NaN
540,532579182624,2022-12-24 06:24:33.536,2022-12-24 06:24:56.095,United States,Apple iPhone,"iPhone14,2",97,2022-12-24 06:24:33.536,ac_content_routine_cancel,English,...,true,true,NaN,true,1.0,1.671821e+09,1.0,1.0,nan,NaN
541,532579182624,2022-12-24 06:24:33.537,2022-12-24 06:24:56.095,United States,Apple iPhone,"iPhone14,2",99,2022-12-24 06:24:33.537,ac_click,English,...,true,true,NaN,true,1.0,1.671821e+09,1.0,1.0,nan,NaN
542,532579182624,2022-12-24 06:24:33.955,2022-12-24 06:24:56.095,United States,Apple iPhone,"iPhone14,2",101,2022-12-24 06:24:33.955,page_view,English,...,true,true,NaN,true,1.0,1.671821e+09,1.0,1.0,nan,NaN
